The following notebook provides the image processing pipeline to obtain a valid region of the lane from an image, in order to be applied to a video.

In [1]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import pickle
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

## Helper functions and classes

In [2]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients for the previous fit
        self.previous_fit = np.array([0,0,0], dtype='float') 
        #polynomial coefficients for the most recent fit
        self.current_fit = np.array([0,0,0], dtype='float') 
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None

In [3]:
def hls_thresholding(warped):
    # Function for image thresholding
    hls = cv2.cvtColor(warped, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    thresh_S=(130,255) # Threshold for the S channel
    thresh_L=(200,255) # Threshold for the L channel
    binary = np.zeros_like(S)
    binary[((L >= thresh_L[0]) & (L <= thresh_L[1])) | ((S >= thresh_S[0]) & (S <= thresh_S[1]))] = 1
    return binary

In [13]:
def img_fit_lane(binary_warped, left_line, right_line, nwindows = 12):
    # Check a first lane fit has been achieved
    if (left_line.bestx is None) & (right_line.bestx is None):
        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
        # Create an output image to draw on and  visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
        
        # Set height of windows
        window_height = np.int(binary_warped.shape[0]/nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
        # Set the width of the windows +/- margin
        margin = 100
        # Set minimum number of pixels found to recenter window
        minpix = 50
        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []
        
        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
                
        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        # Extract left and right line pixel positions and fit a polynomial
        left_line.allx = nonzerox[left_lane_inds]
        left_line.ally = nonzeroy[left_lane_inds]
        left_line.current_fit = np.polyfit(left_line.ally, left_line.allx, 2)
        
        right_line.allx = nonzerox[right_lane_inds]
        right_line.ally = nonzeroy[right_lane_inds]
        right_line.current_fit = np.polyfit(right_line.ally, right_line.allx, 2)
    else:
        # Assume you now have a new warped binary image 
        # from the next frame of video (also called "binary_warped")
        # It's now much easier to find line pixels!
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        margin = 100
        left_lane_inds = ((nonzerox > (left_line.current_fit[0]*(nonzeroy**2) + left_line.current_fit[1]*nonzeroy + 
        left_line.current_fit[2] - margin)) & (nonzerox < (left_line.current_fit[0]*(nonzeroy**2) + 
        left_line.current_fit[1]*nonzeroy + left_line.current_fit[2] + margin))) 

        right_lane_inds = ((nonzerox > (right_line.current_fit[0]*(nonzeroy**2) + right_line.current_fit[1]*nonzeroy + 
        right_line.current_fit[2] - margin)) & (nonzerox < (right_line.current_fit[0]*(nonzeroy**2) + 
        right_line.current_fit[1]*nonzeroy + right_line.current_fit[2] + margin)))  

        # Again, extract left and right line pixel positions and fit a second order polynomial
        left_line.allx = nonzerox[left_lane_inds]
        left_line.ally = nonzeroy[left_lane_inds] 
        left_line.current_fit = np.polyfit(left_line.ally, left_line.allx, 2)
        
        right_line.allx = nonzerox[right_lane_inds]
        right_line.ally = nonzeroy[right_lane_inds]
        right_line.current_fit = np.polyfit(right_line.ally, right_line.allx, 2)

In [14]:
def xval_fitted(image, line_instance, thr=40, nframes = 5):
    # Get pixels of the x coordinate after fitting a polynomial
    ploty = np.linspace(0, image.shape[0]-1, image.shape[0])
    line_fitx = line_instance.current_fit[0]*ploty**2 + line_instance.current_fit[1]*ploty + line_instance.current_fit[2]
    if np.sum(line_instance.previous_fit) == 0:
        line_instance.recent_xfitted.append(line_fitx)
        line_instance.bestx = line_fitx
    else:
        # Compute the differences between the coefficients of the current ad previous frame
        diff_c0 = np.abs(line_instance.current_fit[0] - line_instance.previous_fit[0])
        diff_c1 = np.abs(line_instance.current_fit[1] - line_instance.previous_fit[1])
        diff_c2 = np.abs(line_instance.current_fit[2] - line_instance.previous_fit[2])
        sum_ci = diff_c0 + diff_c1 + diff_c2
        if sum_ci >= thr:
            # If that difference is big the use and average of coefficients of the last nframes
            bestx_last_frames = np.array((line_instance.recent_xfitted[-nframes:]))
            line_instance.bestx = np.mean(bestx_last_frames, axis = 0)
        else:
            # Otherwise use the current fit as the best fit
            line_instance.bestx = line_fitx
        line_instance.recent_xfitted.append(line_fitx)
    line_instance.previous_fit[0] = line_instance.current_fit[0]
    line_instance.previous_fit[1] = line_instance.current_fit[1]
    line_instance.previous_fit[2] = line_instance.current_fit[2]

In [15]:
def get_rad_curvature(image, line_instance):
    # Get the radius of curvature of the line in 'm'
    ploty = np.linspace(0, image.shape[0]-1, image.shape[0])
    y_eval = np.max(ploty)
    line_curverad = ((1 + (2*line_instance.current_fit[0]*y_eval + line_instance.current_fit[1])**2)**1.5) / np.absolute(2*line_instance.current_fit[0])
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Fit new polynomials to x,y in world space
    line_fit_cr = np.polyfit(ploty*ym_per_pix, line_instance.bestx*xm_per_pix, 2)
    # Get the new radii of curvature
    line_instance.radius_of_curvature = ((1 + (2*line_fit_cr[0]*y_eval*ym_per_pix + line_fit_cr[1])**2)**1.5) / np.absolute(2*line_fit_cr[0])

In [16]:
def paint_lane(original_img, binary_warped, Minv, left_line, right_line):
    # Paint the lane region detected over the frames of the video
    ploty = np.linspace(0, original_img.shape[0]-1, original_img.shape[0])
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_line.bestx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_line.bestx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (original_img.shape[1], original_img.shape[0])) 
    # Combine the result with the original image
    avg_curverad = left_line.radius_of_curvature + right_line.radius_of_curvature
    output_img = cv2.addWeighted(original_img, 1, newwarp, 0.3, 0)
    result = cv2.putText(output_img,'Radius of Curvature = %.3f m' %(avg_curverad),(40,100), fontFace=0, fontScale=2, color=(255,255,255), thickness=3)
    return(result)

## Loading parameters

The parameters of the camera are loaded to correct image distortion. These coefficients are obtained and saved on the other Python Notebook.

In [17]:
with open("mtx.pickle", "rb") as f:
    mtx = pickle.load(f)

with open("dist.pickle","rb") as f:
    dist = pickle.load(f)

The mapping between source and destination points to get the top-down view of the road lane. This tranformation is static over all the frames of the video.

In [18]:
# Computing offline the perspective transform
src_points = np.array([(580, 460), (205, 720), (1110, 720), (703, 460)], np.float32)
dst_points = np.array([(320, 0),(320, 720),(960, 720), (960, 0)], np.float32)

# The perspective transform
M = cv2.getPerspectiveTransform(src_points, dst_points)

# ...and the inverse perspective transform
Minv = cv2.getPerspectiveTransform(dst_points, src_points)

## Definition of an image pipeline

In [20]:
left_line =Line()
right_line = Line()

def pipeline(image):
    img = image
    # Undistort image
    undist_img = cv2.undistort(img, mtx, dist, None, mtx)
    
    # Get the shape of the image
    img_shape = (img.shape[1], img.shape[0])
    
    # Project the lane region onto the image plane
    warped = cv2.warpPerspective(undist_img, M, img_shape, flags=cv2.INTER_LINEAR)
    
    # Color thresholding
    binary = hls_thresholding(warped)
    
    # Finding lane
    img_fit_lane(binary, left_line, right_line)
    
    # Fit a polynomial to each line
    xval_fitted(binary, left_line)
    xval_fitted(binary, right_line)
    
    # Get curvature of each line
    get_rad_curvature(binary, left_line)
    get_rad_curvature(binary, right_line)
    
    # Paint lane
    output_img = paint_lane(img, binary, Minv, left_line, right_line)
    return(output_img)

## Video generation

In [21]:
output = 'project_video_output.mp4'
clip = VideoFileClip("project_video.mp4")
white_clip = clip.fl_image(pipeline)
%time white_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4



 24%|██▍       | 302/1261 [00:24<01:16, 12.58it/s]


 48%|████▊     | 604/1261 [00:50<00:59, 11.06it/s]


 72%|███████▏  | 906/1261 [01:16<00:29, 11.99it/s]


 95%|█████████▌| 1200/1261 [01:42<00:05, 11.71it/s]


100%|█████████▉| 1260/1261 [01:47<00:00, 11.62it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

CPU times: user 9min 7s, sys: 1.94 s, total: 9min 9s
Wall time: 1min 47s


In [22]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))